In [1]:
# import streamlit as st
import os
from langchain_community.document_loaders import WebBaseLoader
from langchain_groq import ChatGroq
from langchain.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.retrieval import create_retrieval_chain
from langchain_community.vectorstores import FAISS
import time

from dotenv import load_dotenv
load_dotenv()

True

In [6]:
import json

with open("D:/VARLAB/langchain_tutorial1/data/transcriptions.json","r") as f:
    docs = json.load(f)



In [8]:
type(docs)

dict

In [11]:
# Convert dictionary to text (example format)
text_content = "\n".join([f"{key}: {value}" for key, value in docs.items()])

# Open the file for writing and save the text content
with open("my_dictionary.txt", "w") as f:
    f.write(text_content)




In [12]:
# Load text as Document using TextLoader (assuming the file is saved)
from langchain_community.document_loaders import TextLoader
loader = TextLoader("my_dictionary.txt")
document = loader.load()

In [15]:
##load the Groq API key
groq_api_key=os.environ["GROQ2_API_KEY"]

# if "vector" not in st.session_state:
embeddings=OllamaEmbeddings(model="mxbai-embed-large")


text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
final_documents=text_splitter.split_documents(document)
vectors=FAISS.from_documents(final_documents,embeddings)

In [18]:
import pickle

with open("transcriptions_embeddings.pkl","wb") as f:
    pickle.dump(vectors,f)

In [20]:
with open("transcriptions_embeddings.pkl","rb") as f:
    vectors2=pickle.load(f)

In [14]:
final_documents

[Document(metadata={'source': 'my_dictionary.txt'}, page_content="Unit 10 - Topic 1.webm: welcome to unit ten managing organizational change in this unit we'll discuss an explore how to manage organizational change with a focus on the integration of project management and service management practices as well as considering the human factors of organizational change when you complete this unit you'll be able to describe and discuss the human factors of organizational change and present a change management plan that includes both project management and service management we have three topics in this unit organizational change organizational change models and change model success we are starting a different book for this unit the it in organizational change"),
 Document(metadata={'source': 'my_dictionary.txt'}, page_content="management we have three topics in this unit organizational change organizational change models and change model success we are starting a different book for this uni

In [21]:
# st.title("ChatGroq Demo")
llm = ChatGroq(groq_api_key=groq_api_key,model_name="mixtral-8x7b-32768")

prompt = ChatPromptTemplate.from_template(
    """
    Answer the questions based on the provided context only.
    Look for keywords and exact faq
    Please provide a good response based on the question and context
    <context>
    {context}
    </context>
    Questions:{input}

    """
)

document_chain = create_stuff_documents_chain(llm,prompt)
retriever = vectors.as_retriever()
retriever_chain = create_retrieval_chain(retriever,document_chain)


In [22]:
prompt =input("Input your prompt here")

if prompt:
# start= time.process_time()
    response = retriever_chain.invoke({"input":prompt})
    # print("Response Time: ",time.process_time()-start)
    print(response['answer'])

    # #With a streamlit expander
    # with st.expander("Document Similarity Search"):
    #     #Find the relevant chunks
    # print("Below are to to be ignored. only for displaying similar context")
    print("-"*50,end="\n\n\n")
    for i, doc in enumerate(response["context"]):
        print(doc.page_content)
        print("---------------------------------------------------------")

Based on the provided context, Unit 9 - Topic 3 is not explicitly defined. However, Unit 9 focuses on delivering service management, including general service and technical management practices. Therefore, Unit 9 - Topic 3 would likely involve a deep dive into a specific aspect of service or technical management practices. To get accurate information, it is recommended to refer to the actual course materials or consult the instructor.
--------------------------------------------------


Unit 3.1 - Topic 1 - Project Integration Management.webm: hello and welcome to unit three project management practices in this unit we have two key contact concepts project integration management and managing the triple constraint would you complete this unit you'll will be able to describe project integration and project best practices you'll also be able to to describe develop and demonstrate the use of a project charter and lastly you'll be able to describe and apply the triple constraint of project 

In [1]:
from langchain_community.document_loaders import JSONLoader

In [3]:
loader = JSONLoader(
    file_path='D:/VARLAB/langchain_tutorial1/data/transcriptions.json',
    jq_schema="",
    text_content=False)

data = loader.load()

ImportError: jq package not found, please install it with `pip install jq`

In [4]:
!pip install jq

  Using cached jq-1.7.0.tar.gz (2.0 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
Failed to build jq


  error: subprocess-exited-with-error
  
  × Building wheel for jq (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [5 lines of output]
      running bdist_wheel
      running build
      running build_ext
      Executing: ./configure CFLAGS=-fPIC -pthread --disable-maintainer-mode --with-oniguruma=builtin
      error: [WinError 2] The system cannot find the file specified
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for jq
ERROR: Could not build wheels for jq, which is required to install pyproject.toml-based projects

[notice] A new release of pip is available: 23.2.1 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# To Do

## Use custom Document Loader which will have key as a metadata 